<a href="https://colab.research.google.com/github/Joel-Vijo/MNIST-digit-classifier/blob/main/Dino_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import numpy as np
import random
import matplotlib.pyplot  as plt

In [ ]:
data = open('dinosaurs.csv', 'r').read() 
data = data.lower()
chars = list(set(data))
chars.remove('\n')
chars.sort()
print(chars)
data_size, chars_size = len(data), len(chars)

print("data size= " ,data_size)
print("no of unique chars = " ,chars_size)

char_to_ix = {ch:i for i,ch in enumerate(chars)}   
ix_to_char = {i:ch for i,ch in enumerate(chars)}    
data = data.split('\n')
print(data)
print(list(data[0]))
random.shuffle(data)

['_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
data size=  19656
no of unique chars =  27
['aachenosaurus', 'aardonyx', 'abelisaurus', 'abrictosaurus', 'abrosaurus', 'abydosaurus', 'acantholipan', 'acanthopholis', 'achelousaurus', 'acheroraptor', 'achillesaurus', 'achillobator', 'acristavus', 'acrocanthosaurus', 'acrotholus', 'actiosaurus', 'adamantisaurus', 'adasaurus', 'adelolophus', 'adeopapposaurus', 'adratiklit', 'adynomosaurus', 'aegyptosaurus', 'aeolosaurus', 'aepisaurus', 'aepyornithomimus', 'aerosteon', 'afromimus', 'aetonyx', 'afrovenator', 'agathaumas', 'aggiosaurus', 'agilisaurus', 'agnosphitys', 'agrosaurus', 'agujaceratops', 'agustinia', 'ahshislepelta', 'airakoraptor', 'ajancingenia', 'ajkaceratops', 'akainacephalus', 'alamosaurus', 'alaskacephale', 'albalophosaurus', 'albertaceratops', 'albertadromeus', 'albertavenator', 'albertonykus', 'albertosaurus', 'albinykus', 'alcovasaurus', 

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.input_size = input_size 
        self.output_size = input_size 

        self.i2h = nn.Linear(hidden_size + input_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, input_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input, hidden):

        input_hidd = torch.cat((input,hidden), dim=1)
        hidden = self.i2h(input_hidd)
        hidden = self.sigmoid(hidden)
        output = self.h2o(hidden)
        return hidden , output 

In [ ]:
learning_rate = 0.001

model = RNN(input_size=chars_size, hidden_size=100)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
no_of_epochs=30
hidden_size=100
loss=[]
for i in range(no_of_epochs):
    epoch_loss = 0
    n = 0
    for word in data:
        letters=list(word)
        hidden=torch.zeros([1,hidden_size])
        outputs=torch.zeros([len(letters)+1,chars_size])
        target=list()
        for c in letters:
          target.append(char_to_ix[c])
        target.append(char_to_ix['_'])
        target_tensor=torch.LongTensor(target)
        outputs[0,char_to_ix[letters[0]]]=1
        i=0
        batch_loss=0
        while(i<len(letters)):
          input=torch.zeros([1,chars_size],dtype=torch.float)
          input[0,char_to_ix[letters[i]]]=1
          hidden,output=model(input,hidden)
          outputs[i+1]=output
          i+=1
        batch_loss=criterion(outputs,target_tensor)
        epoch_loss+=batch_loss
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        n+=1
    epoch_loss=epoch_loss/n
    print(epoch_loss.item())
    loss.append(epoch_loss)

2.4331250190734863
2.078227996826172
1.9746249914169312
1.915808081626892
1.8788508176803589
1.8495198488235474
1.823352575302124
1.7993096113204956
1.7769618034362793
1.7559654712677002
1.73612380027771
1.7173817157745361
1.6998182535171509
1.683495283126831
1.6683977842330933
1.6544103622436523
1.6413779258728027
1.6291431188583374
1.6175756454467773
1.6065741777420044
1.5960588455200195
1.5859698057174683
1.5762498378753662
1.5668529272079468
1.5577256679534912
1.5488296747207642
1.5401270389556885
1.5315827131271362
1.5231664180755615
1.5148528814315796


In [ ]:
model.eval()
hidden_size = 100
for letter in chars:
    if(letter!="_"):
      print(letter,"-",end="")
      print(letter,end="")
      char = letter
      input_idx = char_to_ix[char]
      input = torch.zeros([1, chars_size], dtype=torch.float)
      input[0, input_idx] = 1
      

      hidden = torch.zeros([1, hidden_size])
      idx = torch.tensor(char_to_ix[letter])
      i=0
      while ix_to_char[idx.item()] != "_":
          i+=1
          hidden, output = model(input, hidden)
          idx = output.argmax()
          input = torch.zeros([1, chars_size], dtype=torch.float)
          input[0, idx] = 1
          if(ix_to_char[idx.item()]!="_"):
            print(ix_to_char[idx.item()],end="")
      print("")

a -anganosaurus
b -baranosaurus
c -chanosaurus
d -donganosaurus
e -eoroptor
f -fullosaurus
g -gongosaurus
h -huanosaurus
i -ianosaurus
j -jianosaurus
k -kolosaurus
l -longosaurus
m -marosaurus
n -nonosaurus
o -ornitholus
p -palarosaurus
q -quanosaurus
r -raptosaurus
s -saurosaurus
t -talanosaurus
u -uranosaurus
v -velonosaurus
w -wallosaurus
x -xianosaurus
y -yunosaurus
z -zhunosaurus
